In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Packages

In [2]:
import tensorflow as tf

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time
import string
import seaborn as sns
import plotly.graph_objects as go


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
file_path='/content/drive/My Drive/Colab Notebooks/LanguageTranslation/hin.txt'

In [4]:
line = open(file_path, encoding='UTF-8').read().strip().split('\n')
lines=[]
for i in line:
  l=i.split('\t')
  l.pop(2)
  lines.append('\t'.join(l))

In [5]:
lines[:10]

['Wow!\tवाह!',
 'Help!\tबचाओ!',
 'Jump.\tउछलो.',
 'Jump.\tकूदो.',
 'Jump.\tछलांग.',
 'Hello!\tनमस्ते।',
 'Hello!\tनमस्कार।',
 'Cheers!\tवाह-वाह!',
 'Cheers!\tचियर्स!',
 'Got it?\tसमझे कि नहीं?']

# Data Preprocessing

In [14]:
exclude = set(string.punctuation) # Set of all special characters
remove_digits = str.maketrans('', '', string.digits) # Set of all digits

In [15]:
def preprocess_eng_sentence(sent):
    '''Function to preprocess English sentence'''
    sent = sent.lower() # lower casing
    sent = re.sub("'", '', sent) # remove the quotation marks if any
    sent = ''.join(ch for ch in sent if ch not in exclude)
    sent = sent.translate(remove_digits) # remove the digits
    sent = sent.strip()
    sent = re.sub(" +", " ", sent) # remove extra spaces
    sent = '<start> ' + sent + ' <end>' # add <start> and <end> tokens
    return sent

In [16]:
def preprocess_mar_sentence(sent):
    '''Function to preprocess Hindi sentence'''
    sent = re.sub("'", '', sent) # remove the quotation marks if any
    sent = ''.join(ch for ch in sent if ch not in exclude)
    sent = re.sub("[२३०८१५७९४६]", "", sent) # remove the digits
    sent = sent.strip()
    sent = re.sub(" +", " ", sent) # remove extra spaces
    sent = '<start> ' + sent + ' <end>' # add <start> and <end> tokens
    return sent

In [18]:
sent_pairs = []
for line in lines:
    sent_pair = []
    eng, mar = line.split('\t')
    eng = preprocess_eng_sentence(eng)
    sent_pair.append(eng)
    mar = preprocess_mar_sentence(mar)
    sent_pair.append(mar)
    sent_pairs.append(sent_pair)
sent_pairs[2000:2010]

[['<start> the children were well looked after <end>',
  '<start> बच्चों की देखभाल अच्छी तरह से की जाती थी। <end>'],
 ['<start> the doctor advised him not to smoke <end>',
  '<start> डॉक्टर ने उसे सिगरेट न पीने की सलह दी। <end>'],
 ['<start> the doctor advised him not to smoke <end>',
  '<start> डॉक्टर ने उन्हें सिगरेट न पीने की सलह दी। <end>'],
 ['<start> the law says that all men are equal <end>',
  '<start> क़ानून के मुताबिक सभी मनुष्य समान हैं। <end>'],
 ['<start> the plan has been successful so far <end>',
  '<start> प्लान अब तक कामयाब रहा है। <end>'],
 ['<start> the plane will take off in one hour <end>',
  '<start> विमान एक घंटे में उड़ान भरेगा। <end>'],
 ['<start> the wind scattered the leaves about <end>',
  '<start> हवा ने पत्तियों को यहाँवहाँ बिखेर दिया। <end>'],
 ['<start> there are many old temples in kyoto <end>',
  '<start> क्योटो में बहुत सारे पुराने मंदिर हैं। <end>'],
 ['<start> this hat does match the brown dress <end>',
  '<start> यह टोपी उस ब्राऊन ड्रेस के साथ जचती

In [19]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()

        self.create_index()

    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))

        self.vocab = sorted(self.vocab)

        self.word2idx['<pad>'] = 0
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1

        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [20]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [45]:
def load_dataset(pairs, num_examples):
    # pairs => already created cleaned input, output pairs

    # index language using the class defined above    
    inp_lang = LanguageIndex(en for en, ma in pairs)
    targ_lang = LanguageIndex(ma for en, ma in pairs)
    
    # Vectorize the input and target languages
    
    # English sentences
    input_tensor = [[inp_lang.word2idx[s] for s in en.split(' ')] for en, ma in pairs]
    
    # Hindi sentences
    target_tensor = [[targ_lang.word2idx[s] for s in ma.split(' ')] for en, ma in pairs]
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

In [46]:
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(sent_pairs, len(lines))

In [47]:
#Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1, random_state = 101)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(2495, 2495, 278, 278)

In [48]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [49]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
  
      return tf.keras.layers.GRU(units, 
                                  return_sequences=True, 
                                  return_state=True, 
                                  recurrent_activation='sigmoid', 
                                  recurrent_initializer='glorot_uniform')

# Encoder Model

In [50]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

# Decoder Model

In [51]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V 
        #using Bahdanau's Attention
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [52]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [53]:
optimizer=tf.optimizers.Adam()


def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [54]:
checkpoint_dir = '/content/drive/My Drive/Colab Notebooks/LanguageTranslation/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [55]:

EPOCHS = 50

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every epoch
    checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.2997
Epoch 1 Loss 1.9086
Time taken for 1 epoch 17.08180046081543 sec

Epoch 2 Batch 0 Loss 1.8157
Epoch 2 Loss 1.6233
Time taken for 1 epoch 17.26825499534607 sec

Epoch 3 Batch 0 Loss 1.5406
Epoch 3 Loss 1.5203
Time taken for 1 epoch 17.180356979370117 sec

Epoch 4 Batch 0 Loss 1.4251
Epoch 4 Loss 1.4297
Time taken for 1 epoch 17.331021547317505 sec

Epoch 5 Batch 0 Loss 1.3367
Epoch 5 Loss 1.3570
Time taken for 1 epoch 16.894749879837036 sec

Epoch 6 Batch 0 Loss 1.2269
Epoch 6 Loss 1.2924
Time taken for 1 epoch 16.790273666381836 sec

Epoch 7 Batch 0 Loss 1.0850
Epoch 7 Loss 1.2221
Time taken for 1 epoch 17.538662672042847 sec

Epoch 8 Batch 0 Loss 1.0557
Epoch 8 Loss 1.1559
Time taken for 1 epoch 16.92556095123291 sec

Epoch 9 Batch 0 Loss 1.0603
Epoch 9 Loss 1.0851
Time taken for 1 epoch 16.90142583847046 sec

Epoch 10 Batch 0 Loss 1.1913
Epoch 10 Loss 1.0101
Time taken for 1 epoch 17.125291109085083 sec

Epoch 11 Batch 0 Loss 0.9555
Epoch 11 Loss 0.9367
Ti

In [38]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint_dir = '/content/drive/My Drive/Colab Notebooks/LanguageTranslation/training_checkpoints/ckpt-43.index'
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))#

# Evaluation

In [56]:

def evaluate(inputs, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = ''
    for i in inputs[0]:
        if i == 0:
            break
        sentence = sentence + inp_lang.idx2word[i] + ' '
    sentence = sentence[:-1]
    
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [57]:
def predict_random_val_sentence():
    actual_sent = ''
    k = np.random.randint(len(input_tensor_val))
    random_input = input_tensor_val[k]
    random_output = target_tensor_val[k]
    random_input = np.expand_dims(random_input,0)
    result, sentence, attention_plot = evaluate(random_input, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
    print('Input: {}'.format(sentence[8:-6]))
    print('Predicted translation: {}'.format(result[:-6]))
    for i in random_output:
        if i == 0:
            break
        actual_sent = actual_sent + targ_lang.idx2word[i] + ' '
    actual_sent = actual_sent[8:-7]
    print('Actual translation: {}'.format(actual_sent))
    attention_plot = attention_plot[:len(result.split(' '))-2, 1:len(sentence.split(' '))-1]
    sentence, result = sentence.split(' '), result.split(' ')
    sentence = sentence[1:-1]
    result = result[:-2]
    
    # use plotly to generate the heat map
    trace = go.Figure(go.Heatmap(z = attention_plot, x = sentence, y = result, colorscale='Reds'))
    trace.show(renderer="colab")

In [58]:
predict_random_val_sentence()

Input: we drink our tea with sugar
Predicted translation: हम चाय चीनी के साथ लेते हैं। 
Actual translation: हम चाय चीनी डालकर पीते हैं।


In [59]:
predict_random_val_sentence()

Input: you have to go
Predicted translation: तुम्हें जाना होगा। 
Actual translation: तुम्हें जाना होगा।


In [60]:
predict_random_val_sentence()

Input: this hat does match the brown dress
Predicted translation: बच्चों को दवाईयों से बंद है। 
Actual translation: यह टोपी उस ब्राऊन ड्रेस के साथ जचती नहीं है।


In [61]:
predict_random_val_sentence()

Input: can you tell me what size this is
Predicted translation: क्या यह कि तुम मुझे कैसे से क्यों कर सकते हो 
Actual translation: क्या आप मुझे बता सकते हैं कि यह किस नाप का है


In [62]:
predict_random_val_sentence()

Input: i cant live without you
Predicted translation: मैं तुम्हारे बिना जी नहीं सकता। 
Actual translation: मैं तुम्हारे बगैर जी नहीं सकती।


In [63]:
predict_random_val_sentence()

Input: he can imitate the rest of the family
Predicted translation: मुझे कभीकभी के पास जाने की कोशिश करता है। 
Actual translation: वह अपने सारे घरवालों की नकल उतार सकता है।


In [64]:
predict_random_val_sentence()

Input: the girl who works at the bakery is cute
Predicted translation: कुछ भी पहले आएँगे वह वहाँ गया था 
Actual translation: बेकरी में जो लड़की काम करती है वह बहुत क्यूट है।


In [65]:
predict_random_val_sentence()

Input: are you tired
Predicted translation: तुम बीमार हो 
Actual translation: थक गए हो क्या


In [66]:
predict_random_val_sentence()

Input: i am interested in english
Predicted translation: मैं बड़े किताबें पुस्तकालय से सहमत हूँ। 
Actual translation: मुझे अंग्रेज़ी में दिलचस्पी है।


In [67]:
predict_random_val_sentence()

Input: we can hear the ocean from here
Predicted translation: हमे यहाँ से हफ़्ते में जाकर चीनी 
Actual translation: हमे यहाँ से समुंदर सुनाई देता है।
